### Hyperparameter Tuning

In [1]:
import torch
import time
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, classification_report

from scipy.stats import expon, uniform

torch.manual_seed(42)
np.random.seed(42)

In [3]:
device = torch.device("cpu")    # explicity CPU for this scikit-learn 

In [44]:
X_np, y_np = make_classification(
    n_samples=2000, 
    n_features=20, 
    n_informative=15,
    n_redundant=3, 
    n_clusters_per_class=2,
    n_classes=2, 
    class_sep=0.8,  
    random_state=42
)

In [45]:
X_torch = torch.from_numpy(X_np).float().to(device)
y_torch = torch.from_numpy(y_np).long().to(device)

In [46]:
X_torch.shape, y_torch.shape

(torch.Size([2000, 20]), torch.Size([2000]))

In [47]:
X_train_torch, X_test_torch, y_train_torch, y_test_torch = train_test_split(
    X_torch, y_torch, test_size=0.3, random_state=42, stratify=y_torch.cpu() 
)

In [48]:
X_train_torch.shape, X_test_torch.shape

(torch.Size([1400, 20]), torch.Size([600, 20]))

In [67]:
scaler = StandardScaler()

In [68]:
X_train_scaled = scaler.fit_transform(X_train_torch)
X_test_scaled = scaler.transform(X_test_torch)

#### Basline Model (Default Hyperparameters)

In [69]:
baseline_model = SVC(C=1.0, kernel='linear', random_state=42)
baseline_model.fit(X_train_scaled, y_train_torch)

SVC(kernel='linear', random_state=42)

In [70]:
y_pred_baseline_torch = torch.from_numpy(baseline_model.predict(X_test_scaled))
y_pred_baseline_torch[:5], y_test_torch[:5]

(tensor([1, 1, 0, 1, 0]), tensor([1, 0, 1, 1, 1]))

In [71]:
baseline_accuracy = accuracy_score(y_test_torch, y_pred_baseline_torch)
baseline_accuracy

0.7633333333333333

In [72]:
print(classification_report(y_test_torch, y_pred_baseline_torch))

              precision    recall  f1-score   support

           0       0.78      0.74      0.76       299
           1       0.75      0.79      0.77       301

    accuracy                           0.76       600
   macro avg       0.76      0.76      0.76       600
weighted avg       0.76      0.76      0.76       600



#### Grid Search CV

In [75]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'degree': [2, 3]
}

In [76]:
grid_search = GridSearchCV(
    estimator=SVC(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

In [77]:
print("Starting Grid Search...")
start_time_grid = time.time()
grid_search.fit(X_train_scaled, y_train_torch)
end_time_grid = time.time()
grid_search_time = end_time_grid - start_time_grid
print(f"\nGrid Search completed in {grid_search_time:.2f} seconds.")

Starting Grid Search...
Fitting 5 folds for each of 96 candidates, totalling 480 fits


/home/jung/anaconda3/envs/develop_env/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



Grid Search completed in 26.16 seconds.


In [78]:
grid_search.best_params_, grid_search.best_score_

({'C': 1, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'}, 0.9400000000000001)

In [79]:
best_grid_model = grid_search.best_estimator_

In [80]:
y_pred_grid_torch = best_grid_model.predict(X_test_scaled)
grid_accuracy_test = accuracy_score(y_test_torch, y_pred_grid_torch)
grid_accuracy_test

0.9483333333333334

In [81]:
print(classification_report(y_test_torch, y_pred_grid_torch))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95       299
           1       0.95      0.95      0.95       301

    accuracy                           0.95       600
   macro avg       0.95      0.95      0.95       600
weighted avg       0.95      0.95      0.95       600



#### Randomized Search CV

In [82]:
param_dist = {
    'C': expon(scale=10),  
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'gamma': expon(scale=0.1),  
    'degree': randint(2, 5), 
    'coef0': uniform(-1, 2)  
}

In [83]:
random_search = RandomizedSearchCV(
    estimator=SVC(random_state=42),
    param_distributions=param_dist,
    n_iter=50, 
    cv=5,
    scoring='accuracy',
    verbose=1,
    random_state=42,
    n_jobs=-1
)

In [84]:
print("Starting Randomized Search...")
start_time_random = time.time()
random_search.fit(X_train_scaled, y_train_torch)
end_time_random = time.time()
random_search_time = end_time_random - start_time_random

print(f"\nRandomized Search completed in {random_search_time:.2f} seconds.")

Starting Randomized Search...
Fitting 5 folds for each of 50 candidates, totalling 250 fits

Randomized Search completed in 3.64 seconds.


In [85]:
random_search.best_params_, random_search.best_score_

({'C': 14.794837762415114,
  'coef0': -0.6025686369316552,
  'degree': 4,
  'gamma': 0.16898967774861262,
  'kernel': 'rbf'},
 0.9428571428571427)

In [86]:
best_random_model = random_search.best_estimator_
y_pred_random_torch = best_random_model.predict(X_test_scaled)
random_accuracy_test = accuracy_score(y_test_torch, y_pred_random_torch)
random_accuracy_test

0.9466666666666667

In [87]:
print(classification_report(y_test_torch, y_pred_random_torch))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95       299
           1       0.95      0.95      0.95       301

    accuracy                           0.95       600
   macro avg       0.95      0.95      0.95       600
weighted avg       0.95      0.95      0.95       600



#### Comparison and Conclusion

In [88]:
print("\n\n--- Hyperparameter Tuning Summary ---")
print(f"Baseline SVC Accuracy: {baseline_accuracy:.4f}")

print("\nGrid Search CV:")
print(f"  Best CV Accuracy: {grid_search.best_score_:.4f}")
print(f"  Test Set Accuracy: {grid_accuracy_test:.4f}")
print(f"  Best Parameters: {grid_search.best_params_}")
print(f"  Execution Time: {grid_search_time:.2f} seconds")
num_grid_combinations = 1
for param_values_list in param_grid.values(): # Renamed to avoid conflict
    num_grid_combinations *= len(param_values_list)
print(f"  Models trained (combinations * cv_folds): {num_grid_combinations * grid_search.cv}")


print("\nRandomized Search CV:")
print(f"  Best CV Accuracy: {random_search.best_score_:.4f}")
print(f"  Test Set Accuracy: {random_accuracy_test:.4f}")
print(f"  Best Parameters: {random_search.best_params_}")
print(f"  Execution Time: {random_search_time:.2f} seconds")
print(f"  Models trained (n_iter * cv_folds): {random_search.n_iter * random_search.cv}")




--- Hyperparameter Tuning Summary ---
Baseline SVC Accuracy: 0.7633

Grid Search CV:
  Best CV Accuracy: 0.9400
  Test Set Accuracy: 0.9483
  Best Parameters: {'C': 1, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'}
  Execution Time: 26.16 seconds
  Models trained (combinations * cv_folds): 480

Randomized Search CV:
  Best CV Accuracy: 0.9429
  Test Set Accuracy: 0.9467
  Best Parameters: {'C': 14.794837762415114, 'coef0': -0.6025686369316552, 'degree': 4, 'gamma': 0.16898967774861262, 'kernel': 'rbf'}
  Execution Time: 3.64 seconds
  Models trained (n_iter * cv_folds): 250


1.  **Performance:** Both Grid Search and Randomized Search typically find models that outperform the baseline model.
2.  **Execution Time:** Grid Search is exhaustive and can be slow. Randomized Search is often faster for a fixed budget of iterations.
3.  **Effectiveness:** Grid Search finds the best within its grid. Randomized Search explores more broadly and can find good solutions efficiently.